In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import distance
from scipy.sparse.csgraph import minimum_spanning_tree
import math
from scipy import stats

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# from modules.dp_func import *
from modules.Bhattacharyya_func import Bhattacharyya_bounds, Mahalanobis_upper
from modules.Bhatt_knn_func  import Bhattacharyya_knn_bounds


In [3]:
# Set the mean and covariance matrix for each Gaussian distribution
mean_sep = 2

mean1 = [0, 0, 0 , 0 , 0, 0, 0 , 0]
# covariance1 = np.ones((3,3))
covariance1 = np.identity(8)
# covariance1 = [[1, 0, 0],  [0, 1, 0], [0, 0, 0]

mean2 = [mean_sep, 0, 0 , 0, 0, 0,0, 0 ]
# mean2 = [math.sqrt(1/2), math.sqrt(1/2), 0]

# covariance2 = np.ones((3,3))
covariance2= np.identity(8)

#choose the number of samples per group. 
samples = 200

n0 = samples
n1 = samples

# p, q = .5, .5

In [4]:
def obs_params(data):
    mean = np.mean(data, axis=0)# this getting it by the column I believe this should work for [[x y z] [x y z ]] data
    covar = np.cov(data, rowvar= False)
    return [mean, covar]


In [5]:

bounds_l = []
bounds_u = []
bound_Mah = []

bounds_l_knn = []
bounds_u_knn = []

MC_num = 500

for i in range(MC_num):
    # Generate random samples from the multivariate normal distributions
    data1 = np.random.multivariate_normal(mean1, covariance1, n0)
    data2 = np.random.multivariate_normal(mean2, covariance2, n1)

    params1 = obs_params(data1)
    params2 = obs_params(data2)

        
    lower, upper = Bhattacharyya_bounds(params1, params2)
    
    Mah = Mahalanobis_upper( params1, params2)
    bound_Mah.append(Mah)
    bounds_l.append(lower)
    bounds_u.append(upper)
    
    l, u = Bhattacharyya_knn_bounds(data1, data2)
    bounds_l_knn.append(l)
    bounds_u_knn.append(u)
    
    

print("The Bounds")

lower = np.average(bounds_l)
upper = np.average(bounds_u)
print(lower)
print(upper)

print()
print(np.average(bound_Mah))

print()
lower_k = np.average(bounds_l_knn)
upper_k = np.average(bounds_u_knn)
print(lower_k)
print(upper_k)

The Bounds
0.10499127084337956
0.3055638854210771

0.24616670955716802

0.14705347883138267
0.353664728571644


The Bounds
0.05181870809917191
0.2206286006983264
0.18637529642598355


In [6]:
# print(min(bounds_l))
# print(max(bounds_l))


# print(min(bounds_u))
# print(max(bounds_u))

In [7]:
# ub = stats.describe(bounds_u)

In [8]:
dx = 0.001
x = np.arange(-5, 5 + dx, dx)

f0 = np.exp(-0.5 * (x - 0)**2) / np.sqrt(2 * np.pi)
f1 = np.exp(-0.5 * (x - mean_sep)**2) / np.sqrt(2 * np.pi)

fmin = np.minimum(f0, f1)

BER = 0.5 * np.sum(fmin * dx)

print("BER 2d:", BER)

BER 2d: 0.15865513188023003


In [9]:
BER_lst = np.ones(MC_num) * BER


mse_l = ((BER_lst - bounds_l)**2).mean(axis=0)

mse_u =  ((BER_lst - bounds_u)**2).mean(axis=0)

print(mse_l, mse_u)

0.0031038320446628857 0.02195697562009141
